In [ ]:
import pandas as pd
import js
base_url = (await js.fetch("/")).url
import piplite
await piplite.install(["requests", "pyodide-http"])

In [ ]:
def kql2df(kql, timespan="P1D"):
    return pd.DataFrame(requests.get(f"{base_url}/siemqueryutils/api/v1/globalQuery", params = {"query": kql, "timespan": timespan}).json())

In [ ]:
kql = """
SecurityIncident
| summarize count() by Severity, Status, ProviderName
| order by count_
"""

kql2df(kql)